In [56]:
import pandas as pd
from sodapy import Socrata
import requests
import pymongo
import pprint
import json
import datetime
from pandas.io.json import json_normalize

In [57]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.lacity.org", None)

In [58]:
# Example authenticated client (needed for non-public datasets):
client = Socrata("data.lacity.org",
                 "GvIj0aUYYnBp2YRAZ2OHttpL6",
                 username="jackantonyan@gmail.com",
                 password="Ja142536$")


In [59]:
# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("63jg-8b9z", limit=2)

In [60]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [61]:
# Define database and collection
db = client.Crime
collection = db.Crime_La

In [62]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [63]:
results_df.head()

,dr_no,date_rptd,date_occ,time_occ,area,area_name,rpt_dist_no,part_1_2,crm_cd,crm_cd_desc,...,vict_descent,premis_cd,premis_desc,status,status_desc,crm_cd_1,location,lat,lon,cross_street
0,001307355,2010-02-20T00:00:00.000,2010-02-20T00:00:00.000,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,...,H,501,SINGLE FAMILY DWELLING,AA,Adult Arrest,900,300 E GAGE AV,33.9825,-118.2695,NaN
1,011401303,2010-09-13T00:00:00.000,2010-09-12T00:00:00.000,0045,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,W,101,STREET,IC,Invest Cont,740,SEPULVEDA BL,33.9599,-118.3962,MANCHESTER AV


In [64]:
# converting lat and lat into numeric and date occured into date time
results_df['date_occ'] = pd.to_datetime(results_df['date_occ'])
# results_df['area_name'] = results_df['area_name'].astype('str')
# results_df['crm_cd']= pd.to_numeric(results_df['crm_cd'])
# results_df['time_occ']= pd.to_datetime(results_df['time_occ'])
# results_df['lat']= pd.to_numeric(results_df['lat'])
# results_df['lon']= pd.to_numeric(results_df['lon'])
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   dr_no         2 non-null      object        
 1   date_rptd     2 non-null      object        
 2   date_occ      2 non-null      datetime64[ns]
 3   time_occ      2 non-null      object        
 4   area          2 non-null      object        
 5   area_name     2 non-null      object        
 6   rpt_dist_no   2 non-null      object        
 7   part_1_2      2 non-null      object        
 8   crm_cd        2 non-null      object        
 9   crm_cd_desc   2 non-null      object        
 10  mocodes       2 non-null      object        
 11  vict_age      2 non-null      object        
 12  vict_sex      2 non-null      object        
 13  vict_descent  2 non-null      object        
 14  premis_cd     2 non-null      object        
 15  premis_desc   2 non-null      object        

In [65]:
# results_df['time_occ'] = results_df['time_occ'].dt.strftime('%h:%m')

In [66]:
# idx = pd.to_datetime(results_df['date_occ'])
# results_df = results_df.set_index(results_df['date_occ'])

In [86]:
# New data frame with revised columns
new_results_df=results_df[['date_occ','time_occ','crm_cd','crm_cd_desc','area_name', 'lat', 'lon',]]
new_results_df.head()

,date_occ,time_occ,crm_cd,crm_cd_desc,area_name,lat,lon
0,2010-02-20,1350,900,VIOLATION OF COURT ORDER,Newton,33.9825,-118.2695
1,2010-09-12,0045,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",Pacific,33.9599,-118.3962


In [87]:
new_results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date_occ     2 non-null      datetime64[ns]
 1   time_occ     2 non-null      object        
 2   crm_cd       2 non-null      object        
 3   crm_cd_desc  2 non-null      object        
 4   area_name    2 non-null      object        
 5   lat          2 non-null      object        
 6   lon          2 non-null      object        
dtypes: datetime64[ns](1), object(6)
memory usage: 240.0+ bytes


In [95]:
new_results_df['date_occ'] = new_results_df['date_occ'].astype(str)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [96]:
df_date = new_results_df.set_index("date_occ")

In [97]:
df_date.head()

,time_occ,crm_cd,crm_cd_desc,area_name,lat,lon
date_occ,,,,,,
2010-02-20,1350,900,VIOLATION OF COURT ORDER,Newton,33.9825,-118.2695
2010-09-12,0045,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",Pacific,33.9599,-118.3962


In [98]:
print (df_date.index.name)

date_occ


In [101]:
df_final = df_date.loc[['2010-02-20','2010-09-12']]

In [15]:
# # Initialize PyMongo to work with MongoDBs
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)

In [16]:
# # Define database and collection
# db = client.Crime
# collection = db.Crime_La

In [17]:
k = new_results_df.reset_index().T.to_dict().values()
print(k)

dict_values([{'index': 0, 'date_occ': Timestamp('2010-02-20 00:00:00'), 'time_occ': '1350', 'crm_cd': '900', 'crm_cd_desc': 'VIOLATION OF COURT ORDER', 'area_name': 'Newton', 'lat': '33.9825', 'lon': '-118.2695'}, {'index': 1, 'date_occ': Timestamp('2010-09-12 00:00:00'), 'time_occ': '0045', 'crm_cd': '740', 'crm_cd_desc': 'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)', 'area_name': 'Pacific', 'lat': '33.9599', 'lon': '-118.3962'}])


In [18]:
for i in k:
#     print(i)
    pp= pprint.PrettyPrinter(indent=4)
    pp.pprint(i)

{   'area_name': 'Newton',
    'crm_cd': '900',
    'crm_cd_desc': 'VIOLATION OF COURT ORDER',
    'date_occ': Timestamp('2010-02-20 00:00:00'),
    'index': 0,
    'lat': '33.9825',
    'lon': '-118.2695',
    'time_occ': '1350'}
{   'area_name': 'Pacific',
    'crm_cd': '740',
    'crm_cd_desc': 'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)',
    'date_occ': Timestamp('2010-09-12 00:00:00'),
    'index': 1,
    'lat': '33.9599',
    'lon': '-118.3962',
    'time_occ': '0045'}


In [26]:
# pd.DataFrame(results)

In [27]:
# df_json = new_results_df.to_json(orient='records', date_format ='iso')
# pp.pprint(df_json)


In [21]:
collection_count = db.Crime_La.count()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


In [22]:
# a= collection_count

In [23]:
if collection_count != 0:
    collection.drop()

In [24]:
# Insert dictionary into MongoDB as a document
result1 = k.insert_one(Crime)

AttributeError: 'dict_values' object has no attribute 'insert_one'